# О подборе гиперпараметров в процессе обучения

# Автоматический подбор гиперпараметров модели в пакете Optuna
В этом блокноте мы покажем некоторые приемы автоматического подбора гиперпараметров модели на примере использования пакета Optuna. Давайте установим пакет и проверим, что фреймворк TensorFlow и пакет TensorFlow_Dataset тоже установлены.

In [1]:
!pip install optuna
!pip install numpy tensorflow tensorflow_datasets

     |████████████████████████████████| 308 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 209 kB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 8.2 MB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 18.8 MB/s eta 0:00:01
     |████████████████████████████████| 124 kB 30.3 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 78 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 146 kB 16.4 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 19.8 MB/s eta 0:00:01
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.4.39-cp38-cp38-macosx_11_0_arm64.whl size=1543322 sha256=876488721bd16eb96ff13a76ccd4335893686d7d9c28672bd6934b22f55ffcda
  Stored in directory: /Users/bojcevanton/Library/Caches/pip/wheels/fd/23/d9/83cf0a1

Для того тчобы не отвлекаться от сути происходящего, тестировать пакет Optuna мы будем на том же самом примере, что и ранее — на наборе данных «Ирисы Фишера» и нашей маленькой нейронной сети из трех слоев. Давайте импортируем все необходимые пакеты, в т.ч. Optuna.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import metrics, layers, activations, optimizers, losses
import tensorflow_datasets as tfds

import optuna

Как и ранее, разделим набор данных на две части, одна из которых будет использоваться для обучения, а вторая - для валидации модели, и укажем дополнительные параметры (их мы обсуждали в блокноте о `Keras`).

In [3]:
ds_train, ds_test = tfds.load(
    name='iris',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True
)

input_shape = (4, )  
batch_size = 10      
amount_of_classes = 3

Metal device set to: Apple M1 Pro


2022-07-03 18:45:27.556680: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-03 18:45:27.557458: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Аналогичным образом преобразуем набор данных с помощью one-hot encoding, перемешаем и разделим на кусочки (батчи) одинакового размера.

In [4]:
def make_one_hot(x, y):
    return x, tf.one_hot(y, depth=amount_of_classes)

ds_train = (
    ds_train
    .map(make_one_hot)
    .shuffle(len(ds_train))
    .batch(batch_size, drop_remainder=True)
)
    
ds_test = (
    ds_test
    .map(make_one_hot)
    .batch(batch_size, drop_remainder=True)
)

Далее создадим модель, чтобы посмотреть на качество при сдандартных значениях гиперпараметров. Используем ту же самую модель, что и в предыдущем примере, и обучим ее на протяжении 10 эпох.

In [5]:
model = keras.Sequential()
model.add(layers.Dense(32, input_shape=input_shape, activation='sigmoid'))
model.add(layers.Dense(16, activation='sigmoid'))
model.add(layers.Dense(amount_of_classes, activation=activations.softmax))

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.003),
    loss=losses.CategoricalCrossentropy(),
    metrics=[metrics.CategoricalAccuracy()]
)

history = model.fit(ds_train, epochs=10, validation_data=ds_test, verbose=1)

Epoch 1/10


2022-07-03 18:46:30.328606: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-03 18:46:30.332973: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 1s 18ms/step - loss: 1.4315 - categorical_accuracy: 0.3333 - val_loss: 1.2187 - val_categorical_accuracy: 0.3333
Epoch 2/10
12/12 [==============================] - 0s 8ms/step - loss: 1.1643 - categorical_accuracy: 0.2833 - val_loss: 1.0822 - val_categorical_accuracy: 0.3667
Epoch 3/10
 1/12 [=>............................] - ETA: 0s - loss: 1.0538 - categorical_accuracy: 0.2000

2022-07-03 18:46:31.197545: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


12/12 [==============================] - 0s 9ms/step - loss: 1.0748 - categorical_accuracy: 0.3667 - val_loss: 1.0611 - val_categorical_accuracy: 0.6667
Epoch 4/10
12/12 [==============================] - 0s 9ms/step - loss: 1.0466 - categorical_accuracy: 0.5667 - val_loss: 1.0402 - val_categorical_accuracy: 0.6333
Epoch 5/10
12/12 [==============================] - 0s 9ms/step - loss: 1.0173 - categorical_accuracy: 0.6667 - val_loss: 1.0015 - val_categorical_accuracy: 0.6333
Epoch 6/10
12/12 [==============================] - 0s 8ms/step - loss: 0.9765 - categorical_accuracy: 0.6750 - val_loss: 0.9608 - val_categorical_accuracy: 0.6333
Epoch 7/10
12/12 [==============================] - 0s 8ms/step - loss: 0.9331 - categorical_accuracy: 0.6750 - val_loss: 0.9153 - val_categorical_accuracy: 0.6333
Epoch 8/10
12/12 [==============================] - 0s 9ms/step - loss: 0.8855 - categorical_accuracy: 0.6917 - val_loss: 0.8637 - val_categorical_accuracy: 0.7000
Epoch 9/10
12/12 [=========

Отметьте, какое значение точности на валидационном наборе данных мы получили. Вследствие того, что веса инициируются случайным образом, процесс обучения достаточно стохастичен и случайно можно получить удачный набор весов, который приведет к очень высокой точности с первого раза. Однако, чаще всего точность после подбора гиперпараметров оказывается выше вследствие нахождения более удачного сочетания. Сравните эти значения в конце, после оптимизации, и сделайте соответствующие выводы. Перезапустите блокнот несколько раз и проанализируйте полученные результаты.

Для того, чтобы оптимизировать модель, фреймворку Optuna необходимо создать цель оптимизации — функцию, которую фреймворк будет запускать. Выходом функции должно быть одно число, которое необходимо максимизировать или минимизировать.

Функция должна быть полностью самостоятельной, все необходимые инициализации следует совершать в ней. Стоит также учитывать, что фреймворк Optuna можно использовать с использованием внешней базы данных, тем самым запуская множество процессов, которые будут совершать оптимизацию параллельно. Поэтому не стоит использовать глобальные переменные, если они не являются константами.

Функция принимает на вход единственный параметр — объект `optuna.trial.Trial`, который в дальнейшем в ней и используется. Давайте инициализируем новую модель нейронной сети в данной функции и дадим фреймворку Optuna самому подобрать как количество нейронов в слоях, так и используемые функции активации. Для этого вместо указания конкретного числа нейронов и конкретной функции активации указывается то, как этот параметр можно выбирать — с помощью объекта `trial`. Optuna позволяет выбирать из нескольких видов распределений и разных типов данных, а также, если вам нужно указать несколько доступных вариантов, указывать категориальное равномерное распределение с заранее определенными объектами или значениями.

Для количества нейронов первого слоя укажем возможность выбора любого целого числа в промежутке от 16 до 64, а второго слоя — от 4 до 16. Функции активации в обоих случаях укажем как одну из двух: либо можно выбрать сигмоиду, либо ReLU.

Чтобы продемонстрировать работу с непрерывными величинами, коэффициент скорости обучения тоже объявим гиперпараметром и предложим пакету выбирать его из равномерного распределения на отрезке $[0.001, 0.01]$.

Далее в рамках этой же функции обучим модель и, в качестве финального результата обучения укажем последнюю точность на валидационном наборе данных, которая записана в объекте history.

In [6]:
def optuna_objective(trial: optuna.trial.Trial):
    model = keras.Sequential(
        [
            layers.Dense(
                units=trial.suggest_int('l1_neurons', 16, 64),
                input_shape=input_shape,
                activation=trial.suggest_categorical('l1_activation', ['sigmoid', 'relu'])
            ),
            layers.Dense(
                units=trial.suggest_int('l2_neurons', 4, 16),
                activation=trial.suggest_categorical('l2_activation', ['sigmoid', 'relu'])
            ),
            layers.Dense(amount_of_classes, activation=activations.softmax)
        ]
    )
    
    model.compile(
        optimizer=optimizers.Adam(learning_rate=trial.suggest_uniform('lr', 0.001, 0.01)),
        loss=losses.CategoricalCrossentropy(),
        metrics=[metrics.CategoricalAccuracy()]
    )
    
    history = model.fit(ds_train, epochs=10, validation_data=ds_test, verbose=0)
    accuracy = history.history['val_categorical_accuracy'][-1]
    return accuracy

После определения функции оптимизации необходимо создать исследование, то есть инициализировать специальный объект. В данном случае мы укажем, что хотим максимизировать выход функции.

In [7]:
study = optuna.create_study(direction='maximize')

[I 2022-07-03 18:56:26,764] A new study created in memory with name: no-name-056d6668-15de-422e-85fe-748b5c15e70b


Теперь мы готовы запустить процесс поиска. Для этого осталось лишь указать функцию для оптимизации и количество попыток.

In [8]:
study.optimize(optuna_objective, n_trials=30)

2022-07-03 18:56:54.701811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-03 18:56:55.132586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-07-03 18:56:56,006] Trial 0 finished with value: 0.9333333969116211 and parameters: {'l1_neurons': 32, 'l1_activation': 'relu', 'l2_neurons': 13, 'l2_activation': 'sigmoid', 'lr': 0.007185256683353856}. Best is trial 0 with value: 0.9333333969116211.
2022-07-03 18:56:56.179823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-03 18:56:56.524286: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-07-03 18:56:57,403] Trial 1 finished with value: 0.6333333849906921 and parameters: {'l1_neurons': 38, 'l1_activa

После этого в объекте study будет храниться информация о лучшей попытке и использованных параметрах. Мы можем выбрать эти параметры как оптимальные и использовать в дальнейшем для нашей модели.

In [9]:
study.best_params

{'l1_neurons': 37,
 'l1_activation': 'relu',
 'l2_neurons': 16,
 'l2_activation': 'relu',
 'lr': 0.006647381425654292}

Фреймворк Optuna обладает достаточно большими возможностями, не рассмотренными в данной лекции, которые могут вам пригодиться. Возможно, самая полезная возможность -- это возможность использования внешнего хранилища данных, такого как СУБД MySQL или SQLite. Использование внешнего хранилища позволит вам запустить несколько процессов оптимизации параллельно, а также останавливать и продолжать процесс поиска гиперпараметров.